In [1]:
!pip install autogen autogen-agentchat autogen-ext[openai]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 834.0/834.0 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.7 MB/s eta 0:00:00


In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 44.6 MB/s eta 0:00:00


In [5]:
import asyncio
import datetime
import os
import sys
import requests
import json
import re
import numpy as np
import threading
import time
from typing import Union, List, Dict, Optional, Tuple
from autogen import GroupChat, GroupChatManager, ConversableAgent, UserProxyAgent
from sentence_transformers import SentenceTransformer
import faiss
import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset

os.environ["AUTOGEN_VERBOSE"] = "0"

# --- OpenRouter Configuration ---
# Map agent types to OpenRouter models
AGENT_MODELS = {
    "router": "meta-llama/llama-3.3-70b-instruct",
    "language": "qwen/qwen-2.5-72b-instruct",
    "logic": "deepseek/deepseek-r1-distill-llama-70b",
    "social": "openai/gpt-oss-120b",
    "world": "meta-llama/llama-3.3-70b-instruct",
    "summarizer": "meta-llama/llama-3.3-70b-instruct",
    "metacognitive": "meta-llama/llama-3.3-70b-instruct",
    "verification": "meta-llama/llama-3.3-70b-instruct"
}

# Set API key directly
os.environ["OPENROUTER_API_KEY"] = "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"

# OpenRouter API settings
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    print("Error: OPENROUTER_API_KEY environment variable not set.")
    sys.exit(1)

# Correct base URL for OpenAI client
OPENROUTER_API_BASE = "https://openrouter.ai/api/v1"
OPENROUTER_API_ENDPOINT = OPENROUTER_API_BASE + "/chat/completions"

# ======================
# OpenRouter API Function
# ======================
def query_openrouter(messages: List[Dict[str, str]], model: str) -> str:
    """
    Send a query to the OpenRouter API and retrieve the response.

    This function constructs and sends an HTTP POST request to the OpenRouter API
    with the provided messages and model. It handles errors and parses the response
    to extract the content of the assistant's message.

    Args:
        messages (List[Dict[str, str]]): A list of message dictionaries, each containing 'role' and 'content'.
        model (str): The model identifier to use for the query.

    Returns:
        str: The stripped content of the assistant's response, or an empty string if an error occurs.
    """
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": model,
        "messages": messages,
        "temperature": 0.1,
        "max_tokens": 1024
    }

    try:
        response = requests.post(OPENROUTER_API_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()
        data = response.json()
        return data['choices'][0]['message']['content'].strip()
    except requests.exceptions.RequestException as e:
        print(f"OpenRouter API error: {e}")
        return ""
    except (KeyError, IndexError) as e:
        print(f"Response parsing error: {e}")
        return ""

# ======================
# TAL Cognitive Metrics System
# ======================
class TALMetrics:
    """Computes TAL components as cognitive metrics without training"""

    @staticmethod
    def attentive_variance(summary: torch.Tensor, embeddings: torch.Tensor,
                           motivation: 'MotivationSystem') -> float:
        """
        Computes the reflection balance metric using weighted variance and entropy.

        This metric evaluates how balanced the summary is in reflecting the embeddings,
        weighted by the motivation system's success rate.

        Args:
            summary (torch.Tensor): The summary embedding tensor.
            embeddings (torch.Tensor): The input embeddings tensor.
            motivation (MotivationSystem): The motivation system instance for success rate.

        Returns:
            float: The computed attentive variance metric.
        """
        success_rate = motivation.get_success_rate()
        weights = 1 - success_rate * torch.norm(embeddings, dim=1)
        weights = F.softmax(weights, dim=0)

        dists = torch.norm(summary - embeddings, dim=1)
        weighted_dists = weights * dists
        mean_dist = torch.mean(weighted_dists)
        variance = torch.sum(weights * (dists - mean_dist)**2)
        entropy = -torch.sum(weights * torch.log(weights + 1e-9))
        return (variance + 0.1 * entropy).item()

    @staticmethod
    def criticality(summary: torch.Tensor, verifications: dict) -> float:
        """
        Computes the contradiction avoidance metric based on verifications.

        This metric calculates a penalty if contradictions are detected in verifications
        relative to the summary.

        Args:
            summary (torch.Tensor): The summary embedding tensor.
            verifications (dict): Dictionary of verification data.

        Returns:
            float: The computed criticality metric (0.0 if no issues).
        """
        if not verifications:
            return 0.0

        # Find most relevant verification
        query_embed = summary.detach().numpy().astype(np.float32)
        all_verif_embeds = [v["response_embed"] for v in verifications.values()]
        if not all_verif_embeds:
            return 0.0

        index = faiss.IndexFlatL2(query_embed.shape[1])
        index.add(np.array(all_verif_embeds))
        D, I = index.search(query_embed, 1)

        # Calculate contradiction penalty
        verification = list(verifications.values())[I[0][0]]
        if "contradiction" in verification["verification"].lower():
            response_embed = torch.tensor(verification["response_embed"])
            return (1 - F.cosine_similarity(summary, response_embed, dim=0)).item()
        return 0.0

    @staticmethod
    def temporal_coherence(current_state: torch.Tensor,
                            prev_states: List[torch.Tensor]) -> float:
        """
        Computes the reasoning consistency metric across states.

        This metric measures dissimilarity from the mean of previous states to ensure coherence.

        Args:
            current_state (torch.Tensor): The current hidden state tensor.
            prev_states (List[torch.Tensor]): List of previous hidden state tensors.

        Returns:
            float: The computed temporal coherence metric.
        """
        if len(prev_states) < 2:
            return 0.0
        prev_tensor = torch.stack(prev_states)
        prev_mean = torch.mean(prev_tensor, dim=0)
        return (1 - F.cosine_similarity(current_state, prev_mean, dim=0)).item()

# ======================
# Enhanced UMM Components
# ======================
class LSTM_Memory(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, num_layers: int = 3):
        """
        Initializes the LSTM-based memory module.

        This module processes sequences of embeddings to generate summaries and computes TAL metrics.

        Args:
            input_size (int): The size of input embeddings.
            hidden_size (int): The size of LSTM hidden states.
            num_layers (int, optional): Number of LSTM layers. Defaults to 3.
        """
        super(LSTM_Memory, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, input_size)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedder = SentenceTransformer('all-MiniLM-L6-v2')
        self.history_states = []  # Stores last 5 hidden states
        self.lock = threading.Lock()

    def forward(self, input_seq: List[str]) -> Tuple[torch.Tensor, dict]:
        """
        Performs a forward pass through the LSTM to generate summary and metrics.

        Converts input sequence to embeddings, processes through LSTM, and computes TAL metrics.

        Args:
            input_seq (List[str]): List of input strings to process.

        Returns:
            Tuple[torch.Tensor, dict]: Summary tensor and dictionary of TAL metrics.
        """
        # Convert input sequence to embeddings
        embeddings = self.embedder.encode(input_seq)
        embeddings_tensor = torch.tensor(embeddings, dtype=torch.float32).unsqueeze(0)

        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, 1, self.hidden_size)
        c0 = torch.zeros(self.num_layers, 1, self.hidden_size)

        # LSTM processing
        out, (hn, cn) = self.lstm(embeddings_tensor, (h0, c0))
        summary = self.fc(out[:, -1, :])

        # Compute TAL metrics (no gradients)
        with torch.no_grad():
            metrics = {
                "variance": TALMetrics.attentive_variance(
                    summary,
                    embeddings_tensor[0],
                    motivation_system
                ),
                "criticality": TALMetrics.criticality(
                    summary,
                    long_term_memory.verifications
                ),
                "temporal": TALMetrics.temporal_coherence(
                    hn,
                    self.history_states[-5:]  # Last 5 states
                )
            }

        # Update history states
        with self.lock:
            self.history_states.append(hn.clone().detach())
            if len(self.history_states) > 5:
                self.history_states.pop(0)

        return summary, metrics

class LongTermMemory:
    """Enhanced memory system with TAL cognitive metrics"""

    def __init__(self):
        """
        Initializes the long-term memory system.

        Sets up storage dictionaries, embedder, FAISS index, LSTM model, and thread lock.
        """
        self.agent_profile = {}
        self.user_profiles = {}
        self.domain_knowledge = {}
        self.reflections = []
        self.workflows = {}
        self.verifications = {}
        self.embedder = SentenceTransformer('all-MiniLM-L6-v2')
        self.index = faiss.IndexFlatL2(384)
        self.memory_model = LSTM_Memory(input_size=384, hidden_size=128)
        self.memory_sequence = []
        self.last_metrics = None  # Store last computed metrics
        self.lock = threading.Lock()

    def store(self, category: str, content: dict):
        """
        Stores content in the appropriate memory category.

        Handles different categories like profiles, knowledge, reflections, and verifications.
        For reflections, adds to FAISS index and sequence.

        Args:
            category (str): The category of memory to store in.
            content (dict): The content dictionary to store.
        """
        with self.lock:
            if category == "agent_profile":
                self.agent_profile.update(content)
            elif category == "user_profile":
                user_id = content.get("user_id", "default")
                self.user_profiles.setdefault(user_id, {}).update(content)
            elif category == "domain_knowledge":
                self.domain_knowledge.update(content)
            elif category == "reflection":
                embedding = self.embedder.encode(content['insight'])
                self.index.add(embedding.reshape(1, -1))
                self.reflections.append(content)
                self.memory_sequence.append(content['insight'])
            elif category == "verification":
                task = content.get("task", "unknown")
                # Store response embedding for criticality metric
                content["response_embed"] = self.embedder.encode(content["response"])
                self.verifications[task] = content

    def store_workflow(self, task_type: str, agent_sequence: List[str]):
        """
        Stores a workflow sequence for a given task type.

        Appends the agent sequence to the list of workflows for that type.

        Args:
            task_type (str): The type of task.
            agent_sequence (List[str]): List of agent names in sequence.
        """
        with self.lock:
            self.workflows.setdefault(task_type, []).append(agent_sequence)

    def retrieve(self, query: str) -> str:
        """
        Retrieves relevant memory based on the query.

        Searches FAISS index for reflections or checks domain knowledge.

        Args:
            query (str): The query string to search for.

        Returns:
            str: Retrieved memory content or "No relevant memory found".
        """
        with self.lock:
            if self.index.ntotal > 0:
                query_embedding = self.embedder.encode(query)
                D, I = self.index.search(query_embedding.reshape(1, -1), 1)
                if I[0][0] >= 0:
                    return f"Previous reflection: {self.reflections[I[0][0]]['insight']}"
            for key, value in self.domain_knowledge.items():
                if key.lower() in query.lower():
                    return value
            return "No relevant memory found"

    def generate_summary(self) -> str:
        """
        Generates a summary of reflections with TAL metrics.

        Processes memory sequence through LSTM and finds closest reflection.

        Returns:
            str: Summary string or error message.
        """
        with self.lock:
            if not self.memory_sequence:
                return "No reflections available for summary"

            # Generate summary embedding and TAL metrics
            summary_embedding, metrics = self.memory_model(self.memory_sequence)
            self.last_metrics = metrics  # Store metrics for reporting

            # Print cognitive metrics
            print(f"\n\033[1;36mCOGNITIVE METRICS:\033[0m")
            print(f"  Balance (Variance): {metrics['variance']:.4f}")
            print(f"  Criticality: {metrics['criticality']:.4f}")
            print(f"  Temporal Coherence: {metrics['temporal']:.4f}")

            # Find closest reflection to summary embedding
            D, I = self.index.search(summary_embedding.detach().numpy(), 1)
            if I[0][0] >= 0:
                return f"Summary of reflections: {self.reflections[I[0][0]]['insight']}"
            return "Summary generation failed"

    def get_workflow(self, task_type: str) -> Union[List[str], None]:
        """
        Retrieves the latest workflow for a task type.

        Args:
            task_type (str): The type of task.

        Returns:
            Union[List[str], None]: Latest agent sequence or None if not found.
        """
        with self.lock:
            if task_type in self.workflows and self.workflows[task_type]:
                return self.workflows[task_type][-1]
            return None

class MotivationSystem:
    """Dynamic motivation system with feedback loop"""

    def __init__(self):
        """
        Initializes the motivation system.

        Sets up drives, goals, triggers, progress, outcomes, and thread lock.
        """
        self.long_term_drives = []
        self.short_term_goals = []
        self.reactive_triggers = {}
        self.progress = {}
        self.recent_outcomes = []
        self.success_count = 0
        self.failure_count = 0
        self.lock = threading.Lock()

    def add_long_term_drive(self, drive: str):
        """
        Adds a long-term drive to the system.

        Args:
            drive (str): The drive description.
        """
        with self.lock:
            self.long_term_drives.append(drive)

    def set_short_term_goal(self, goal: str):
        """
        Sets a short-term goal with initial progress of 0.0.

        Args:
            goal (str): The goal description.
        """
        with self.lock:
            self.short_term_goals.append(goal)
            self.progress[goal] = 0.0

    def update_goal_progress(self, goal: str, progress: float):
        """
        Updates the progress of a short-term goal, clamped between 0.0 and 1.0.

        Args:
            goal (str): The goal to update.
            progress (float): The new progress value.
        """
        with self.lock:
            if goal in self.progress:
                self.progress[goal] = max(0.0, min(1.0, progress))

    def add_reactive_trigger(self, trigger: str, response: str):
        """
        Adds a reactive trigger and its response.

        Args:
            trigger (str): The trigger condition.
            response (str): The response action.
        """
        with self.lock:
            self.reactive_triggers[trigger] = response

    def update_from_outcome(self, task: str, success: bool):
        """
        Updates the system based on a task outcome.

        Adjusts recent outcomes, progress, and counts accordingly.

        Args:
            task (str): The task description.
            success (bool): Whether the outcome was successful.
        """
        with self.lock:
            self.recent_outcomes.append(success)
            if len(self.recent_outcomes) > 10:
                self.recent_outcomes.pop(0)
            if success:
                self.progress["Provide accurate responses"] = min(1.0, self.progress.get("Provide accurate responses", 0) + 0.2)
                self.success_count += 1
            else:
                self.set_short_term_goal(f"Improve accuracy on {task}")
                self.failure_count += 1

    def get_success_rate(self):
        """
        Calculates the success rate from recent outcomes.

        Returns:
            float: Success rate (1.0 if no outcomes).
        """
        with self.lock:
            if not self.recent_outcomes:
                return 1.0
            return sum(self.recent_outcomes) / len(self.recent_outcomes)

    def get_current_motivation(self) -> str:
        """
        Generates a string representation of current motivations.

        Includes long-term drives and short-term goals with progress bars.

        Returns:
            str: Formatted motivation string.
        """
        with self.lock:
            motivation = ""
            if self.long_term_drives:
                motivation += "Long-term objectives:\n" + "\n".join(f"- {d}" for d in self.long_term_drives)
            if self.short_term_goals:
                motivation += "\n\nShort-term goals:\n"
                for goal in self.short_term_goals:
                    progress = self.progress.get(goal, 0.0)
                    progress_bar = f"[{'█' * int(progress*10)}{'░' * (10 - int(progress*10))}]"
                    motivation += f"- {goal} {progress_bar} {int(progress*100)}%\n"
            return motivation

# Initialize UMM components
long_term_memory = LongTermMemory()
motivation_system = MotivationSystem()

# Populate initial knowledge
long_term_memory.store("domain_knowledge", {"grammar": "Study of language structure rules"})
long_term_memory.store("domain_knowledge", {"logic": "Systematic reasoning and inference"})
long_term_memory.store("domain_knowledge", {"emotions": "Complex psychological states"})
long_term_memory.store("domain_knowledge", {"tokyo": "Capital of Japan with rich history"})
long_term_memory.store("domain_knowledge", {"mathematics": "Study of numbers, quantities, and shapes, including arithmetic, algebra, geometry, and calculus"})
long_term_memory.store("domain_knowledge", {"history": "Chronological record of past events, causes, and effects"})
long_term_memory.store("domain_knowledge", {"geography": "Study of Earth's physical features, climates, and human populations"})
long_term_memory.store("domain_knowledge", {"science": "Systematic enterprise that builds and organizes knowledge in the form of testable explanations and predictions"})
long_term_memory.store("domain_knowledge", {"probability": "Branch of mathematics dealing with uncertainty and random events"})
long_term_memory.store("domain_knowledge", {"statistics": "Collection, analysis, interpretation, and presentation of data"})
motivation_system.add_long_term_drive("Excel at analytical tasks")
motivation_system.add_long_term_drive("Provide helpful and accurate information across domains")
motivation_system.add_long_term_drive("Improve problem-solving efficiency over time")
motivation_system.add_long_term_drive("Integrate knowledge from multiple domains for comprehensive solutions")
motivation_system.set_short_term_goal("Provide accurate responses")
motivation_system.set_short_term_goal("Solve mathematical problems step-by-step")
motivation_system.set_short_term_goal("Verify facts before incorporation")
motivation_system.set_short_term_goal("Adapt to new task requirements dynamically")
motivation_system.add_reactive_trigger("error", "Attempt self-correction")
motivation_system.add_reactive_trigger("inconsistency", "Reconcile conflicting information")
motivation_system.add_reactive_trigger("incomplete_analysis", "Seek additional domain input")
motivation_system.add_reactive_trigger("low_confidence", "Request verification or additional data")

# ======================
# Snapshot Memory System
# ======================
class SnapshotMemory:
    """Simulates human visual snapshot memory for structured information"""

    def __init__(self, memory: LongTermMemory):
        """
        Initializes the snapshot memory system.

        Sets up templates, FAISS index for visuals, embedder, snapshots storage, and lock.

        Args:
            memory (LongTermMemory): The long-term memory instance.
        """
        self.memory = memory
        self.page_templates = {}
        self.visual_index = faiss.IndexFlatL2(384)  # For visual pattern matching
        self.embedder = SentenceTransformer('all-MiniLM-L6-v2')
        self.snapshots = {}  # Stores visual snapshots with metadata
        self.lock = threading.Lock()
        print("\033[1;32mSnapshot Memory System initialized\033[0m")

    def create_snapshot(self, content: str, structure_type: str = "text") -> dict:
        """
        Creates a visual snapshot representation of the content.

        Extracts structural features, creates embedding, and stores in index and snapshots.

        Args:
            content (str): The content to snapshot.
            structure_type (str, optional): Type of structure. Defaults to "text".

        Returns:
            dict: The created snapshot dictionary.
        """
        # Extract structural patterns
        lines = content.split('\n')
        line_count = len(lines)
        avg_line_length = sum(len(line) for line in lines) / max(1, line_count)

        # Identify key structural elements
        structural_features = {
            "line_count": line_count,
            "avg_line_length": avg_line_length,
            "indentation_pattern": self._extract_indentation_pattern(lines),
            "section_headers": self._extract_section_headers(lines),
            "bullet_points": self._count_bullet_points(lines),
            "numerical_patterns": self._extract_numerical_patterns(content)
        }

        # Create visual embedding
        visual_embedding = self._create_visual_embedding(structural_features, content)

        snapshot = {
            "timestamp": datetime.datetime.now().isoformat(),
            "content": content,
            "structural_features": structural_features,
            "visual_embedding": visual_embedding,
            "structure_type": structure_type
        }

        # Store snapshot
        with self.lock:
            snapshot_id = f"snapshot_{len(self.snapshots)}"
            self.snapshots[snapshot_id] = snapshot
            self.visual_index.add(np.array([visual_embedding]))

        return snapshot

    def _extract_indentation_pattern(self, lines: List[str]) -> List[int]:
        """
        Extracts the indentation pattern from lines.

        Args:
            lines (List[str]): List of content lines.

        Returns:
            List[int]: List of indentation levels.
        """
        pattern = []
        for line in lines:
            indent = len(line) - len(line.lstrip())
            pattern.append(indent)
        return pattern

    def _extract_section_headers(self, lines: List[str]) -> List[str]:
        """
        Identifies section headers based on patterns.

        Args:
            lines (List[str]): List of content lines.

        Returns:
            List[str]: List of identified headers.
        """
        headers = []
        header_patterns = [r'^#+ ', r'^[A-Z][A-Z\s]+:$', r'^[IVX]+\.']  # Markdown, caps, roman numerals

        for line in lines:
            for pattern in header_patterns:
                if re.match(pattern, line.strip()):
                    headers.append(line.strip())
                    break
        return headers

    def _count_bullet_points(self, lines: List[str]) -> int:
        """
        Counts bullet points in the content.

        Args:
            lines (List[str]): List of content lines.

        Returns:
            int: Number of bullet points.
        """
        bullet_count = 0
        bullet_patterns = [r'^[\*\-] ', r'^\d+\.', r'^[a-z]\)']  # *, -, numbers, letters

        for line in lines:
            for pattern in bullet_patterns:
                if re.match(pattern, line.strip()):
                    bullet_count += 1
                    break
        return bullet_count

    def _extract_numerical_patterns(self, content: str) -> List[float]:
        """
        Extracts numerical values from the content.

        Args:
            content (str): The content string.

        Returns:
            List[float]: List of extracted numbers.
        """
        numbers = []
        number_patterns = [r'\d+\.\d+', r'\d+']

        for pattern in number_patterns:
            matches = re.findall(pattern, content)
            numbers.extend([float(match) for match in matches])
        return numbers

    def _create_visual_embedding(self, structural_features: dict, content: str) -> np.array:
        """
        Creates a visual embedding representing the structural layout.

        Args:
            structural_features (dict): Extracted structural features.
            content (str): The original content.

        Returns:
            np.array: The embedding array.
        """
        # Create a text representation of structural features
        structure_text = f"""
        Line count: {structural_features['line_count']}
        Average line length: {structural_features['avg_line_length']}
        Headers: {', '.join(structural_features['section_headers'])}
        Bullet points: {structural_features['bullet_points']}
        Numbers: {structural_features['numerical_patterns']}
        """

        # Combine with content for embedding
        combined = structure_text + "\n" + content
        return self.embedder.encode(combined)

    def recall_snapshot(self, query: str, similarity_threshold: float = 0.7) -> List[dict]:
        """
        Recalls snapshots similar to the query based on embedding similarity.

        Args:
            query (str): The query string.
            similarity_threshold (float, optional): Distance threshold for similarity. Defaults to 0.7.

        Returns:
            List[dict]: List of similar snapshots with similarity scores.
        """
        query_embedding = self.embedder.encode(query).reshape(1, -1)

        with self.lock:
            if self.visual_index.ntotal == 0:
                return []

            # Search for similar snapshots
            D, I = self.visual_index.search(query_embedding, min(5, self.visual_index.ntotal))

            # Filter by similarity threshold
            similar_snapshots = []
            for i, distance in zip(I[0], D[0]):
                if i < len(self.snapshots) and distance < similarity_threshold:
                    snapshot_id = list(self.snapshots.keys())[i]
                    similar_snapshots.append({
                        "snapshot": self.snapshots[snapshot_id],
                        "similarity": 1 - distance  # Convert distance to similarity
                    })

            return similar_snapshots

    def get_structural_template(self, content: str) -> dict:
        """
        Extracts a structural template from the content.

        Args:
            content (str): The content to extract from.

        Returns:
            dict: Template with type and features.
        """
        snapshot = self.create_snapshot(content)
        return {
            "structure_type": snapshot["structure_type"],
            "structural_features": snapshot["structural_features"]
        }

    def apply_template(self, new_content: str, template: dict) -> str:
        """
        Applies a structural template to new content.

        Currently supports text type; extend for more types.

        Args:
            new_content (str): The new content to format.
            template (dict): The template to apply.

        Returns:
            str: Formatted content.
        """
        if template["structure_type"] == "text":
            return self._format_as_text(new_content, template["structural_features"])
        # Add more template types as needed
        return new_content

    def _format_as_text(self, content: str, features: dict) -> str:
        """
        Formats content according to text template features.

        Applies indentation pattern to lines.

        Args:
            content (str): The content to format.
            features (dict): Structural features.

        Returns:
            str: Formatted content string.
        """
        lines = content.split('\n')
        formatted_lines = []

        # Apply indentation pattern if available
        indentation_pattern = features.get("indentation_pattern", [])
        for i, line in enumerate(lines):
            if i < len(indentation_pattern):
                indent = " " * indentation_pattern[i]
                formatted_lines.append(indent + line.strip())
            else:
                formatted_lines.append(line)

        return "\n".join(formatted_lines)

# Initialize snapshot memory
snapshot_memory = SnapshotMemory(long_term_memory)

# ======================
# Spontaneous Thought System (Fixed with threading)
# ======================
class SpontaneousThoughtAgent:
    def __init__(self, memory: LongTermMemory, motivation: MotivationSystem):
        """
        Initializes the spontaneous thought agent.

        Starts a background thread for periodic insight generation.

        Args:
            memory (LongTermMemory): The memory system.
            motivation (MotivationSystem): The motivation system.
        """
        self.memory = memory
        self.motivation = motivation
        self.last_activation = datetime.datetime.now()
        self.insight_count = 0
        self.activation_log = []
        self._stop_event = threading.Event()
        self.thread = threading.Thread(target=self.run, daemon=True)
        self.thread.start()
        print("\033[1;32mSpontaneous Thought Agent started\033[0m")

    def run(self):
        """
        Runs the background loop for periodic activations.
        """
        while not self._stop_event.is_set():
            time.sleep(120)  # Wait 2 minutes between activations
            if self.should_activate():
                self.generate_insight()

    def should_activate(self) -> bool:
        """
        Determines if the agent should activate based on time since last activation.

        Returns:
            bool: True if should activate, False otherwise.
        """
        time_since_last = (datetime.datetime.now() - self.last_activation).total_seconds()
        return time_since_last > 60  # Activate every 60+ seconds

    def generate_insight(self):
        """
        Generates and stores a random insight as a reflection.
        """
        insights = [
            "Periodic reflection enhances problem-solving effectiveness",
            "Cognitive diversity improves solution quality",
            "Metacognition enables adaptive learning strategies",
            "Temporal coherence strengthens memory consolidation",
            "Balancing exploration and exploitation optimizes learning",
            "Contextual awareness improves decision accuracy",
            "Error analysis drives cognitive improvement",
            "Cross-domain knowledge integration fosters innovation"
        ]
        insight = np.random.choice(insights)
        keywords = re.findall(r'\b\w+\b', insight.lower())
        reflection = {
            "timestamp": datetime.datetime.now().isoformat(),
            "insight": insight,
            "keywords": keywords
        }
        self.memory.store("reflection", reflection)
        self.last_activation = datetime.datetime.now()
        self.insight_count += 1
        self.activation_log.append({
            "timestamp": reflection["timestamp"],
            "insight": insight
        })
        print(f"\n\033[1;35mSPONTANEOUS INSIGHT:\033[0m {insight}")

    def stop(self):
        """
        Stops the background thread.
        """
        self._stop_event.set()
        self.thread.join(timeout=1.0)

# Start spontaneous thought agent
spontaneous_agent = SpontaneousThoughtAgent(long_term_memory, motivation_system)

# ======================
# Rigorous Reasoning Framework
# ======================
RIGOROUS_PROMPT_TEMPLATE = """
### Core Instructions for Rigorous Analysis ###

*   **Rigor is Paramount:** Your primary goal is to produce a complete and rigorously justified analysis. Ensure every step is logically sound, supported by evidence or clear reasoning, and free from assumptions unless explicitly stated.
*   **Honesty About Completeness:** Present only significant partial insights that you can rigorously justify. If information is missing, note it explicitly and suggest how to obtain it.
*   **Step-by-Step Reasoning:** Break down the problem into clear, sequential steps. Justify each step with references to definitions, theorems, rules, or factual knowledge where applicable.
*   **Precision in Language:** Use precise terminology. Avoid ambiguity. Define terms if they might be unclear.
*   **Error Handling:** Anticipate potential errors or misconceptions and address them proactively.

### Output Format ###

Your response MUST be structured into:
1. Summary: Verdict and method sketch. The verdict should be a concise statement of the conclusion. The method sketch should outline the key approach in 1-2 sentences.
2. Detailed Analysis: Step-by-step reasoning with clear justifications. Number each step and explain why it follows from the previous one.
3. Final Answer: Clearly boxed answer using format: #### <numerical_value> or appropriate format for non-numerical answers.

### Few-Shot Examples ###

Example 1:
Task: Calculate 2 + 3.
1. Summary: Verdict: The sum is 5. Method Sketch: Basic arithmetic addition of two integers.
2. Detailed Analysis: Step 1: Identify the operands as integers 2 and 3. Step 2: Apply the addition operator, which combines their values: 2 + 3 = 5. Justification: Addition is defined as the total when combining quantities.
3. Final Answer: #### 5

Example 2:
Task: Solve for x in x - 5 = 10.
1. Summary: Verdict: x = 15. Method Sketch: Algebraic manipulation by adding 5 to both sides.
2. Detailed Analysis: Step 1: Start with the equation x - 5 = 10. Step 2: Add 5 to both sides to isolate x: x - 5 + 5 = 10 + 5, simplifying to x = 15. Justification: Addition property of equality maintains equation balance.
3. Final Answer: #### 15

Example 3:
Task: If a train travels 60 km/h for 2.5 hours, how far does it go?
1. Summary: Verdict: 150 km. Method Sketch: Use distance formula: distance = speed × time.
2. Detailed Analysis: Step 1: Recall the formula for distance: d = s × t, where s is speed and t is time. Justification: Derived from definition of speed as distance over time. Step 2: Plug in values: s = 60 km/h, t = 2.5 h, so d = 60 × 2.5 = 150 km. Step 3: Verify units: km/h × h = km, consistent.
3. Final Answer: #### 150

Example 4:
Task: What is the area of a rectangle with length 4 and width 5?
1. Summary: Verdict: 20 square units. Method Sketch: Apply area formula for rectangle: area = length × width.
2. Detailed Analysis: Step 1: Recall rectangle area formula: A = l × w. Justification: Based on geometry where area is the product of perpendicular dimensions. Step 2: Substitute l = 4, w = 5: A = 4 × 5 = 20. Step 3: Confirm units are squared if applicable.
3. Final Answer: #### 20

Example 5:
Task: Determine if 9 is a prime number.
1. Summary: Verdict: No, 9 is not prime. Method Sketch: Check divisibility by integers less than its square root.
2. Detailed Analysis: Step 1: Definition: A prime number is greater than 1 and has no divisors other than 1 and itself. Step 2: Square root of 9 is 3, check divisors 2 and 3. Step 3: 9 ÷ 3 = 3 (integer), so divisible by 3. Justification: Violates prime definition.
3. Final Answer: #### no

Example 6:
Task: Simplify the fraction 8/12.
1. Summary: Verdict: 2/3. Method Sketch: Divide numerator and denominator by greatest common divisor.
2. Detailed Analysis: Step 1: Find GCD of 8 and 12, which is 4. Justification: Euclidean algorithm. Step 2: Divide numerator: 8 ÷ 4 = 2; denominator: 12 ÷ 4 = 3. Step 3: Verify simplified form has GCD 1.
3. Final Answer: #### 2/3
"""

VERIFICATION_SYSTEM_PROMPT = """
### Verification Principles ###
1. Identify all issues in the provided analysis, including factual inaccuracies, logical flaws, mathematical errors, or incomplete reasoning.
2. Classify issues as Critical Errors (e.g., wrong conclusion, major miscalculation) or Justification Gaps (e.g., missing steps, unstated assumptions).
3. Output structured verification report: List critical errors first, then gaps, then overall assessment.
4. Verify final answer format: #### <numerical_value>. Check if it's present and correctly formatted.
5. Suggest corrections or improvements where possible.
6. Be objective and evidence-based in your verification.

### Few-Shot Examples ###

Example 1:
Problem: 2 + 2 = ?
Solution: #### 5
Verification Report:
- Critical Errors: Incorrect arithmetic result (2 + 2 = 4, not 5). Suggestion: Recheck basic addition.
- Justification Gaps: No detailed steps or explanation provided.
- Overall Assessment: Invalid solution.
- Format: Correct, but content wrong.

Example 2:
Problem: x + 3 = 7
Solution: x = 4 #### 4
Verification Report:
- Critical Errors: None.
- Justification Gaps: Missing explicit step of subtracting 3 from both sides. Suggestion: Add algebraic justification.
- Overall Assessment: Correct conclusion, but incomplete reasoning.
- Format: Correct.

Example 3:
Problem: Area of circle with radius 5.
Solution: 78.5 #### 78.5
Verification Report:
- Critical Errors: None (using π ≈ 3.14).
- Justification Gaps: Formula A = πr² not stated; π value not specified. Suggestion: Explicitly state formula and approximation used.
- Overall Assessment: Accurate, but lacks transparency.
- Format: Correct.

Example 4:
Problem: Simplify √16.
Solution: 4 #### 4
Verification Report:
- Critical Errors: None.
- Justification Gaps: No mention that square root is the principal root or handling of negative root. Suggestion: Note that √ denotes positive root.
- Overall Assessment: Correct for principal square root.
- Format: Correct.

Example 5:
Problem: Is 1 a prime number?
Solution: No #### no
Verification Report:
- Critical Errors: None.
- Justification Gaps: Missing definition of prime (greater than 1, no divisors other than 1 and itself). Suggestion: Include full definition.
- Overall Assessment: Correct, but justification incomplete.
- Format: Correct.

Example 6:
Problem: Calculate 10% of 200.
Solution: 20 #### 20
Verification Report:
- Critical Errors: None.
- Justification Gaps: No explanation of percentage as division by 100. Suggestion: Show step: (10/100) × 200 = 20.
- Overall Assessment: Accurate result, enhance with steps.
- Format: Correct.
"""

DOMAIN_PROMPTS = {
    "language": """
    Apply rigorous linguistic analysis to the task. Focus on grammar, syntax, semantics, pragmatics, or etymology as relevant. Use standard linguistic terminology and cite standard rules or examples from established sources like English grammar rules.

    Example 1 (Grammar):
    Task: "Correct the sentence: 'She don't like apples.'"
    Response: "The error is subject-verb agreement. 'She' is third-person singular, requiring 'doesn't'. Corrected: 'She doesn't like apples.' Justification: English subject-verb agreement rule. #### doesn't"

    Example 2 (Semantics):
    Task: "Explain the difference between 'affect' and 'effect'"
    Response: "'Affect' is typically a verb meaning to influence something. 'Effect' is typically a noun meaning the result of a change. Example: Rain affects plants (verb). The effect of rain is growth (noun). Justification: Based on standard English usage dictionaries like Oxford. #### affect:verb, effect:noun"

    Example 3 (Part of Speech):
    Task: "Identify the part of speech for 'running' in 'The running water'."
    Response: "'Running' functions as an adjective modifying 'water' (gerund form used adjectivally). Justification: In this context, it's not a verb but descriptive. #### adjective"

    Example 4 (Syntax):
    Task: "Parse the sentence: 'The quick brown fox jumps over the lazy dog.'"
    Response: "Subject: 'The quick brown fox' (noun phrase). Verb: 'jumps'. Prepositional phrase: 'over the lazy dog'. Justification: Basic English sentence structure S-V-O with modifiers. #### subject: fox, verb: jumps"

    Example 5 (Pragmatics):
    Task: "What is implied in 'Can you pass the salt?'"
    Response: "This is a polite request rather than a question about ability. Justification: Pragmatic implication in English conversational implicature (Grice's maxims). #### request"
    """,

    "logic": """
    Use formal logical reasoning with clear premises and conclusions. For mathematical tasks, show all steps, formulas, and calculations. Reference mathematical theorems or logical principles where applicable.

    Example 1 (Mathematics):
    Task: "Solve for x: 2(x + 3) = 16"
    Response: "Step 1: Distribute: 2x + 6 = 16. Step 2: Subtract 6: 2x = 10. Step 3: Divide by 2: x = 5. Justification: Distributive property and inverse operations. #### 5"

    Example 2 (Deductive Reasoning):
    Task: "If all A are B, and some B are C, are all A C?"
    Response: "No, this is invalid syllogism. Premises: ∀A → B, ∃B → C. Conclusion doesn't follow. Example: All dogs (A) are mammals (B), some mammals (B) are cats (C), but no dogs are cats. Justification: Logical quantifiers. #### no"

    Example 3 (Probability):
    Task: "Probability of rolling a 6 on a fair die."
    Response: "Favorable outcomes: 1 (the 6). Total outcomes: 6. Probability = 1/6 ≈ 0.1667. Justification: Classical probability definition. #### 1/6"

    Example 4 (Set Theory):
    Task: "Union of sets {1,2} and {2,3}."
    Response: "Union includes all unique elements: {1,2,3}. Justification: Set union operation A ∪ B. #### {1,2,3}"

    Example 5 (Inductive Reasoning):
    Task: "Pattern: 1, 4, 9, 16... Next number?"
    Response: "Squares: 1²=1, 2²=4, 3²=9, 4²=16. Next: 5²=25. Justification: Inductive pattern recognition. #### 25"
    """,

    "social": """
    Analyze psychological and social dynamics with evidence-based reasoning. Reference psychological theories, studies, or established concepts like Maslow's hierarchy or social identity theory.

    Example 1 (Emotional Intelligence):
    Task: "How might someone feel if they were excluded from a group activity?"
    Response: "Possible feelings: rejection, loneliness, sadness, anger. Justification: Humans have fundamental need for belonging (Maslow's hierarchy); exclusion triggers social pain similar to physical pain (Eisenberger studies). #### rejection, loneliness, sadness"

    Example 2 (Social Dynamics):
    Task: "What factors contribute to group cohesion?"
    Response: "Factors: Shared goals, interpersonal attraction, optimal group size (3-7), external threats, frequent interaction. Justification: Social identity theory (Tajfel) emphasizes in-group favoritism; Tuckman's stages show cohesion in 'norming' phase. #### shared goals, interpersonal attraction, group size, external threats"

    Example 3 (Bias):
    Task: "Explain confirmation bias."
    Response: "Tendency to seek/interpret information confirming preexisting beliefs. Example: Ignoring evidence against political views. Justification: Cognitive psychology (Kahneman); leads to echo chambers. #### seeking confirming evidence"

    Example 4 (Motivation):
    Task: "What motivates employee performance?"
    Response: "Intrinsic (autonomy, mastery) and extrinsic (rewards, recognition). Justification: Self-Determination Theory (Deci & Ryan) balances both for optimal motivation. #### intrinsic, extrinsic"

    Example 5 (Conflict Resolution):
    Task: "How to resolve interpersonal conflict?"
    Response: "Steps: Active listening, identify needs, find common ground, compromise. Justification: Thomas-Kilmann model; collaborative style yields best long-term results. #### active listening, compromise"
    """,

    "world": """
    Provide historically and factually accurate analysis with proper context. Cite reliable sources or general knowledge facts. Include timelines, geographical details, or cultural contexts where relevant.

    Example 1 (Geography):
    Task: "What are the major geographical features of Japan?"
    Response: "Archipelago with 6,852 islands; 4 main: Honshu, Hokkaido, Kyushu, Shikoku. Mountainous (73% land), volcanic (Ring of Fire), earthquakes common. Limited arable land (12%). Justification: USGS data on tectonics. #### mountainous, volcanic, archipelago"

    Example 2 (History):
    Task: "What were the main causes of World War I?"
    Response: "Long-term: Militarism (arms race), Alliances (Triple Entente vs Alliance), Imperialism (colonial rivalries), Nationalism (ethnic tensions). Trigger: Assassination of Archduke Franz Ferdinand (1914). Justification: Historical analysis (e.g., Fischer thesis on German aggression). #### militarism, alliances, imperialism, nationalism"

    Example 3 (Culture):
    Task: "Describe traditional Japanese tea ceremony."
    Response: "Chanoyu: Ritual preparation/serving of matcha. Emphasizes harmony, respect, purity, tranquility. Steps: Guest reception, tea whisking, serving. Justification: Rooted in Zen Buddhism, 16th century origins (Sen no Rikyū). #### harmony, respect, purity, tranquility"

    Example 4 (Economics):
    Task: "What is GDP?"
    Response: "Gross Domestic Product: Total value of goods/services produced in a country over time. Nominal vs real (inflation-adjusted). Justification: Standard economic measure (World Bank definitions). #### total economic output"

    Example 5 (Science):
    Task: "Explain photosynthesis."
    Response: "Process: Plants convert CO2 + H2O + light → glucose + O2. Equation: 6CO2 + 6H2O → C6H12O6 + 6O2. Justification: Biological process in chloroplasts; Calvin cycle details. #### CO2 to glucose"
    """,

    "collaboration": """
    Coordinate with other experts to integrate diverse perspectives. Ensure the integrated analysis is cohesive, highlights synergies or conflicts between domains, and builds a comprehensive view.

    Example Integration 1:
    Task: "Analyze the impact of climate change on coastal communities"
    Response: "Environmental (world: rising sea levels erode coastlines), Economic (logic: cost of relocation), Social (social: displacement causes stress). Comprehensive: Requires policy integrating all. Justification: IPCC reports on multifaceted impacts. #### environmental, economic, social"

    Example Integration 2:
    Task: "Interpret the proverb 'A stitch in time saves nine'"
    Response: "Language: Metaphor for timely action. Logic: Preventive maintenance reduces future costs. Social: Encourages proactive behavior in relationships. Integrated: Timely intervention prevents escalation across domains. #### timely action prevents worse"

    Example Integration 3:
    Task: "Evaluate remote work effects."
    Response: "Social: Isolation vs flexibility (work-life balance). World: Global adoption post-COVID. Logic: Productivity metrics vary. Integrated: Benefits outweigh if managed (hybrid models). Justification: Studies (Gallup on engagement). #### flexibility, isolation"

    Example Integration 4:
    Task: "Assess AI in education."
    Response: "Logic: Algorithms personalize learning. Social: Reduces teacher-student interaction. Language: Improves literacy tools. World: Accessibility in developing countries. Integrated: Transformative if ethically deployed. #### personalization, ethics"

    Example Integration 5:
    Task: "Analyze vaccine hesitancy."
    Response: "Social: Fear from misinformation. World: Historical mistrust (e.g., Tuskegee). Logic: Risk-benefit calculation flawed. Language: Framing affects perception. Integrated: Multi-pronged approach needed. #### misinformation, mistrust"
    """
}

# ======================
# Classification System
# ======================
def classify_task(task: str) -> List[str]:
    """
    Classifies the task into required expertise domains using a prompt.

    Falls back to keyword matching if no domains detected.

    Args:
        task (str): The task description.

    Returns:
        List[str]: List of domain names.
    """
    classification_prompt = f"""
    {RIGOROUS_PROMPT_TEMPLATE}
    {DOMAIN_PROMPTS["collaboration"]}

    ### Classification Task ###
    Analyze the following task to determine required expertise domains:
    - "language": Grammar, syntax, semantics, linguistic analysis, word meanings, sentence structure.
    - "logic": Mathematical calculations, logical reasoning, puzzles, deductive/inductive inference.
    - "social": Human emotions, psychology, social dynamics, interpersonal relations, behavioral analysis.
    - "world": Factual knowledge, history, geography, science facts, current events, cultural contexts.

    Task: {task}

    Provide your classification in the structured format:
    1. Summary:
        a. Verdict: Required domains (comma-separated)
        b. Method Sketch: Brief justification for each domain
    2. Detailed Analysis: Step-by-step reasoning on why each domain applies or not.

    ### Few-Shot Examples ###
    Task: Solve 2+2.
    1. Summary: Verdict: logic. Method Sketch: Involves arithmetic operation.
    2. Detailed Analysis: Step 1: Task requires calculation. Step 2: Matches logic domain for math.

    Task: Grammar check sentence.
    1. Summary: Verdict: language. Method Sketch: Applies linguistic rules.
    2. Detailed Analysis: Step 1: Focus on syntax and grammar. Step 2: No math or social elements.

    Task: Historical event date.
    1. Summary: Verdict: world. Method Sketch: Recalls factual historical knowledge.
    2. Detailed Analysis: Step 1: Requires timeline facts. Step 2: Fits world domain.

    Task: What is the capital of Japan?
    1. Summary: Verdict: world. Method Sketch: Geographical fact recall.
    2. Detailed Analysis: Step 1: Involves location knowledge. Step 2: No reasoning or emotion needed.

    Task: How would a person feel if their friend betrayed them?
    1. Summary: Verdict: social. Method Sketch: Emotional and psychological analysis.
    2. Detailed Analysis: Step 1: Deals with human feelings. Step 2: Matches social domain.

    Task: Solve the puzzle: If A is B and B is C, is A C?
    1. Summary: Verdict: logic. Method Sketch: Logical deduction required.
    2. Detailed Analysis: Step 1: Transitive property test. Step 2: Pure reasoning.

    Task: Correct 'Their going to the store.'
    1. Summary: Verdict: language. Method Sketch: Homophone correction.
    2. Detailed Analysis: Step 1: Identifies 'Their' vs 'They're'. Step 2: Linguistic error.

    Task: Causes of climate change.
    1. Summary: Verdict: world. Method Sketch: Scientific and environmental facts.
    2. Detailed Analysis: Step 1: Involves global knowledge. Step 2: No personal emotions.
    """

    messages = [
        {"role": "system", "content": classification_prompt},
        {"role": "user", "content": "Perform domain classification"}
    ]

    response = query_openrouter(messages, model=AGENT_MODELS["router"])

    # Improved domain extraction
    domains = []
    if "language" in response.lower():
        domains.append("language")
    if "logic" in response.lower():
        domains.append("logic")
    if "social" in response.lower():
        domains.append("social")
    if "world" in response.lower():
        domains.append("world")

    # Fallback to classification if no domains found
    if not domains:
        print("No domains detected in classification, using fallback logic")
        if "grammar" in task.lower() or "sentence" in task.lower() or "linguistic" in task.lower():
            domains.append("language")
        if "equation" in task.lower() or "solve" in task.lower() or "math" in task.lower():
            domains.append("logic")
        if "emotional" in task.lower() or "social" in task.lower() or "jealous" in task.lower():
            domains.append("social")
        if "history" in task.lower() or "tokyo" in task.lower() or "context" in task.lower():
            domains.append("world")

    return domains if domains else ["complex"]

# ======================
# Enhanced Thought Prompt with Snapshot Memory
# ======================
def build_thought_prompt(task: str, context: str = "", domain: str = "", collaborators: List[str] = []) -> str:
    """
    Builds a thought prompt for agents based on various components.

    Incorporates motivation, memory, domain prompts, rigorous template, collaboration, and task.

    Args:
        task (str): The original task.
        context (str, optional): Additional context. Defaults to "".
        domain (str, optional): The domain for specific prompts. Defaults to "".
        collaborators (List[str], optional): List of collaborator names. Defaults to [].

    Returns:
        str: The constructed prompt string.
    """
    memory_recall = long_term_memory.retrieve(task)
    motivation = motivation_system.get_current_motivation()
    domain_specific = DOMAIN_PROMPTS.get(domain, DOMAIN_PROMPTS["collaboration"])

    # Recall relevant snapshots
    similar_snapshots = snapshot_memory.recall_snapshot(task)
    snapshot_context = ""
    if similar_snapshots:
        best_snapshot = max(similar_snapshots, key=lambda x: x["similarity"])
        snapshot_context = f"\n### Relevant Structural Memory ###\nPrevious similar task structure: {best_snapshot['snapshot']['structural_features']}"

    collaborator_str = ", ".join(collaborators) if collaborators else "working independently"

    # Add answer format examples for math problems
    format_examples = ""
    if "logic" in domain:
        format_examples = (
            "\n\n### Answer Format Examples ###\n"
            "1. After reasoning: Therefore, the final answer is 42. #### 42\n"
            "2. After calculation: The total cost is $156.75. #### 156.75\n"
            "3. After deduction: Mary has 5 apples left. #### 5\n"
            "4. For fractions: The simplified form is 3/4. #### 3/4\n"
            "5. For sets: The intersection is {1,3}. #### {1,3}"
        )

    prompt = f"""
    ## Unified Mind Model Thought Structure

    ### Motivation System ###
    {motivation}

    ### Related Memory ###
    {memory_recall}
    {snapshot_context}

    ### Domain Expertise ###
    {domain_specific}

    {RIGOROUS_PROMPT_TEMPLATE}

    ### Collaboration Context ###
    Collaborating with: {collaborator_str}

    ### Task Context ###
    {context}

    ### Original Task ###
    {task}
    {format_examples}
    """
    return prompt.strip()

# ======================
# Enhanced Reflection with Snapshot Memory
# ======================
def perform_reflection(task: str, response: str, agent_sequence: List[str]):
    """
    Reflects on task performance, creates snapshot, and stores reflection and workflow.

    Args:
        task (str): The task description.
        response (str): The response generated.
        agent_sequence (List[str]): Sequence of agents used.
    """
    # Create snapshot of the solution
    snapshot = snapshot_memory.create_snapshot(response, "solution")

    insight = f"Completed task: {task}. Approach was effective. Structural pattern: {snapshot['structural_features']}"
    keywords = ["success", "effective", "approach", "structural_pattern"]
    reflection = {
        "timestamp": datetime.datetime.now().isoformat(),
        "task": task,
        "response": response,
        "insight": insight,
        "keywords": keywords,
        "snapshot_id": list(snapshot_memory.snapshots.keys())[-1]  # Reference to the snapshot
    }
    long_term_memory.store("reflection", reflection)

    task_type = "collaboration"
    long_term_memory.store_workflow(task_type, agent_sequence)

# ======================
# Verification System
# ======================
def verify_solution(problem: str, solution: str) -> str:
    """
    Verifies a solution using the verification prompt and stores the result.

    Args:
        problem (str): The original problem.
        solution (str): The proposed solution.

    Returns:
        str: The verification report text.
    """
    prompt = f"""
    {VERIFICATION_SYSTEM_PROMPT}

    ### Problem ###
    {problem}

    ### Solution ###
    {solution}

    Apply verification principles to the solution above.
    """

    messages = [
        {"role": "system", "content": VERIFICATION_SYSTEM_PROMPT},
        {"role": "user", "content": prompt}
    ]

    verification_text = query_openrouter(messages, model=AGENT_MODELS["verification"])

    # Store verification with response embedding for TAL
    long_term_memory.store("verification", {
        "task": problem,
        "response": solution,
        "verification": verification_text,
        "timestamp": datetime.datetime.now().isoformat()
    })

    return verification_text

# ======================
# Agent Configuration
# ======================
def create_rigorous_agent(name: str, domain: str) -> ConversableAgent:
    """
    Creates a ConversableAgent for a specific domain.

    Args:
        name (str): The agent name.
        domain (str): The domain for model selection.

    Returns:
        ConversableAgent: The created agent.
    """
    model = AGENT_MODELS[domain]
    return ConversableAgent(
        name=name,
        system_message="",
        llm_config={
            "config_list": [{
                "model": model,
                "api_key": OPENROUTER_API_KEY,
                "base_url": OPENROUTER_API_BASE
            }]
        },
        description=f"{domain.capitalize()} expert agent",
    )

# Initialize agents
router_agent = ConversableAgent(
    name="router_agent",
    system_message=(
        "Analyze tasks and identify required domains. "
        "Output ONLY a comma-separated list of domain names from: language, logic, social, world.\n"
        "Example: 'language,logic'\n\n"
        "DO NOT include any analysis or additional text in your response."
    ),
    llm_config={
        "config_list": [{
            "model": AGENT_MODELS["router"],
            "api_key": OPENROUTER_API_KEY,
            "base_url": OPENROUTER_API_BASE
        }]
    },
    description="Task routing agent",
)

summarizer_agent = ConversableAgent(
    name="summarizer_agent",
    system_message=(
        "Integrate contributions into coherent summary. "
        "Maintain rigorous structure and preserve key insights. "
        "Highlight connections between different expert analyses. Ensure the summary is comprehensive, resolves any conflicts, and provides a unified conclusion.\n"
        "### FORMAT REQUIREMENT ###\n"
        "1. Always conclude with the final answer using format: #### <numerical_value>\n"
        "2. Ensure answer is boxed at the end of your response\n"
        "3. Examples:\n"
        "   - Mathematical: 'Therefore, the final answer is 42. #### 42'\n"
        "   - Financial: 'The total cost is $156.75. #### 156.75'\n"
        "   - Logical: 'The conclusion is true. #### true'\n"
        "   - Multiple choice: 'The correct option is B. #### B'\n"
        "   - Textual: 'The grammatical error is subject-verb agreement. #### subject-verb agreement'\n"
        "\n### Integration Examples ###\n"
        "Example 1 (Multiple Domains):\n"
        "Language Expert: 'The sentence contains a dangling modifier'\n"
        "Logic Expert: 'The reasoning follows valid logical structure'\n"
        "Integrated: 'While the logical structure is valid (Logic), the expression contains a grammatical issue (Language). The complete analysis reveals both structural validity and surface errors. #### valid logic, grammatical error'\n"
        "\nExample 2 (Mathematical):\n"
        "Logic Expert: 'The equation can be solved through algebraic manipulation'\n"
        "World Expert: 'The context suggests real-world application of the formula'\n"
        "Integrated: 'The algebraic solution (Logic) has practical applications in the specified domain (World). The complete solution is x = 5. #### 5'\n"
        "\nExample 3 (Social and World):\n"
        "Social Expert: 'The event could cause widespread anxiety and social unrest'\n"
        "World Expert: 'The historical context is the Cold War era with nuclear tensions'\n"
        "Integrated: 'The historical period (World) could induce specific emotional responses like fear (Social). Overall impact: Heightened global tension. #### Cold War, anxiety'\n"
        "\nExample 4 (Language and Logic):\n"
        "Language Expert: 'The riddle uses word play on homophones'\n"
        "Logic Expert: 'Solving requires lateral thinking'\n"
        "Integrated: 'Combining linguistic ambiguity (Language) with deductive steps (Logic) yields the answer. #### word play, lateral thinking'\n"
        "\nExample 5 (All Domains):\n"
        "Language: 'Metaphorical language'\n"
        "Logic: 'Valid argument structure'\n"
        "Social: 'Evokes empathy'\n"
        "World: 'Based on historical event'\n"
        "Integrated: 'The narrative uses metaphor (Language) in a logical plot (Logic) to evoke social emotions (Social) grounded in history (World). #### metaphorical, logical, empathetic, historical'"
    ),
    llm_config={
        "config_list": [{
            "model": AGENT_MODELS["summarizer"],
            "api_key": OPENROUTER_API_KEY,
            "base_url": OPENROUTER_API_BASE
        }]
    },
    description="Summary integration agent",
)

language_agent = create_rigorous_agent("language_agent", "language")
logic_agent = create_rigorous_agent("logic_agent", "logic")
social_agent = create_rigorous_agent("social_agent", "social")
world_agent = create_rigorous_agent("world_agent", "world")

metacognitive_agent = ConversableAgent(
    name="metacognitive_agent",
    system_message=(
        f"{VERIFICATION_SYSTEM_PROMPT}\n\n"
        "### Collaboration Oversight ###\n"
        "As the metacognitive agent, your role is to oversee the entire collaborative process in this multi-agent system. You must evaluate the current state of the group chat, including all previous messages, expert analyses, summaries, and verifications. Use metacognitive principles such as self-reflection, adaptive decision-making, and awareness of the system's cognitive processes to determine the next step.\n\n"
        "Key Responsibilities:\n"
        "1. Assess Completeness: Evaluate if the task has been fully addressed with rigorous, evidence-based analysis, consistent conclusions, and no remaining ambiguities or errors.\n"
        "2. Detect Issues: Identify any logical inconsistencies, unresolved conflicts between experts, incomplete reasoning, factual errors, or gaps in domain coverage.\n"
        "3. Monitor Progress: Consider the overall flow, ensuring efficient collaboration without unnecessary cycles.\n"
        "4. Decide Action: Based on your assessment, choose one of the following options:\n"
        "   - TERMINATE: If the task is completely resolved: The analysis is rigorous, all domains are appropriately covered, verification shows no critical errors or major gaps, the summary is cohesive, and the final answer is accurate and well-justified. No further input is needed.\n"
        "   - CONTINUE: If refinement is required: There are unresolved conflicts, justification gaps, minor errors, or areas needing deeper exploration within the current domains. Allow another round of collaboration or summarization.\n"
        "   - ROUTER: If re-routing is necessary: The current domains are insufficient or mismatched for the task, new domains are needed, or the initial classification was incorrect. Send back to the router for re-assignment.\n\n"
        "Best Practices:\n"
        "- Be objective: Base decisions on evidence from the chat history.\n"
        "- Promote efficiency: Avoid unnecessary continuation; terminate when criteria are met.\n"
        "- Ensure adaptability: Use ROUTER if the task evolves or requires different expertise.\n\n"
        "Output ONLY: TERMINATE, CONTINUE, or ROUTER. Do not provide explanations or additional text.\n\n"
        "### Few-Shot Examples ###\n\n"
        "Example 1 (TERMINATE):\n"
        "Chat History Summary: Logic expert solved the equation correctly. Verification: No errors, full justification. Summary: Cohesive, final answer matches.\n"
        "Output: TERMINATE\n\n"
        "Example 2 (CONTINUE):\n"
        "Chat History Summary: Social and world experts disagree on impact interpretation. Verification: Justification gap in reconciling views. Summary: Incomplete resolution.\n"
        "Output: CONTINUE\n\n"
        "Example 3 (ROUTER):\n"
        "Chat History Summary: Task involves linguistic analysis, but only logic and world domains active. Verification: Missing domain coverage.\n"
        "Output: ROUTER\n\n"
        "Example 4 (TERMINATE):\n"
        "Chat History Summary: All experts contributed, analyses align, verification clean, summary comprehensive.\n"
        "Output: TERMINATE\n\n"
        "Example 5 (CONTINUE):\n"
        "Chat History Summary: Minor factual error in world expert's input, not addressed in summary. Verification: Critical error detected.\n"
        "Output: CONTINUE\n\n"
        "Example 6 (ROUTER):\n"
        "Chat History Summary: Task shifted to require social dynamics, but current agents are language and logic only.\n"
        "Output: ROUTER\n\n"
        "Example 7 (CONTINUE):\n"
        "Chat History Summary: Summary has unresolved ambiguity in final conclusion. Verification: Gap in evidence.\n"
        "Output: CONTINUE\n\n"
        "Example 8 (TERMINATE):\n"
        "Chat History Summary: Multi-domain integration successful, no conflicts, rigorous steps throughout.\n"
        "Output: TERMINATE"
    ),
    llm_config={
        "config_list": [{
            "model": AGENT_MODELS["metacognitive"],
            "api_key": OPENROUTER_API_KEY,
            "base_url": OPENROUTER_API_BASE
        }]
    },
    description="Metacognitive oversight agent",
)

# ======================
# Collaborative Group Chat
# ======================
agents = [router_agent, summarizer_agent, language_agent,
          logic_agent, social_agent, world_agent, metacognitive_agent]
specialized_agents = {
    "language": language_agent,
    "logic": logic_agent,
    "social": social_agent,
    "world": world_agent
}

# State management
collaboration_state = {}

def custom_speaker_selection(last_speaker: ConversableAgent, groupchat: GroupChat) -> Union[ConversableAgent, str, None]:
    """
    Custom speaker selection logic for the group chat.

    Manages agent turns based on state, routing, collaboration, summarization, and metacognition.

    Args:
        last_speaker (ConversableAgent): The last speaking agent.
        groupchat (GroupChat): The group chat instance.

    Returns:
        Union[ConversableAgent, str, None]: Next speaker, "auto", or None to terminate.
    """
    agents_by_name = {agent.name: agent for agent in groupchat.agents}
    messages = groupchat.messages
    task = messages[0]["content"] if messages else ""

    # Initialize collaboration state
    if task not in collaboration_state:
        collaboration_state[task] = {
            "context": "",
            "active_agents": [],
            "completed_agents": [],
            "round": 0
        }

    state = collaboration_state[task]

    # Record agent activation
    if last_speaker.name != "initiator" and last_speaker.name and messages:
        state["context"] += f"\n\n[{last_speaker.name}]: {messages[-1]['content']}"

    # Initial routing
    if not messages or last_speaker.name == "initiator":
        return router_agent

    # After router identifies required agents
    if last_speaker.name == "router_agent" and messages:
        router_response = messages[-1]["content"].strip()
        domain_list = [d for d in ["language", "logic", "social", "world"] if d in router_response.lower()]
        if not domain_list:
            domain_list = classify_task(task)
        state["active_agents"] = [specialized_agents[d] for d in domain_list if d in specialized_agents]
        if state["active_agents"]:
            state["round"] = 1
            for agent in state["active_agents"]:
                domain = [k for k, v in specialized_agents.items() if v == agent][0]
                collaborators = [a.name for a in state["active_agents"] if a != agent]
                prompt = build_thought_prompt(
                    task=task,
                    context=state["context"],
                    domain=domain,
                    collaborators=collaborators
                )
                agent.update_system_message(prompt)
            return state["active_agents"][0]
        return "auto"

    # During collaboration rounds
    if last_speaker in state["active_agents"]:
        if last_speaker not in state["completed_agents"]:
            state["completed_agents"].append(last_speaker)
        next_agents = [a for a in state["active_agents"] if a not in state["completed_agents"]]
        if next_agents:
            agent = next_agents[0]
            domain = [k for k, v in specialized_agents.items() if v == agent][0]
            collaborators = [a.name for a in state["active_agents"] if a != agent]
            prompt = build_thought_prompt(
                task=task,
                context=state["context"],
                domain=domain,
                collaborators=collaborators
            )
            agent.update_system_message(prompt)
            return agent
        state["round"] += 1
        state["completed_agents"] = []
        return summarizer_agent

    # After summarization
    if last_speaker.name == "summarizer_agent" and messages:
        state["context"] += f"\n\n[SUMMARY]: {messages[-1]['content']}"
        motivation_system.update_goal_progress("Provide accurate responses", 0.5)
        return metacognitive_agent

    # After metacognitive assessment
    if last_speaker.name == "metacognitive_agent" and messages:
        content = messages[-1]["content"].strip().upper()
        decision = None
        for option in ["TERMINATE", "CONTINUE", "ROUTER"]:
            if option in content:
                decision = option
                break
        if decision == "TERMINATE":
            motivation_system.update_goal_progress("Provide accurate responses", 1.0)
            return None
        elif decision == "CONTINUE":
            state["round"] += 1
            if state["active_agents"]:
                for agent in state["active_agents"]:
                    domain = [k for k, v in specialized_agents.items() if v == agent][0]
                    collaborators = [a.name for a in state["active_agents"] if a != agent]
                    prompt = build_thought_prompt(
                        task=task,
                        context=state["context"],
                        domain=domain,
                        collaborators=collaborators
                    )
                    agent.update_system_message(prompt)
                return state["active_agents"][0]
        elif decision == "ROUTER":
            state["active_agents"] = []
            state["completed_agents"] = []
            state["round"] = 0
            return router_agent
        return "auto"

    return "auto"

# ======================
# Task Processing with Snapshot Memory
# ======================
def process_task(task: str) -> str:
    """
    Processes a task through the collaborative group chat system.

    Handles state, chat initiation, response extraction, reflection, verification, and outcome update.

    Args:
        task (str): The task to process.

    Returns:
        str: The final response.
    """
    if task in collaboration_state:
        del collaboration_state[task]

    # Check for similar snapshots first
    similar_snapshots = snapshot_memory.recall_snapshot(task)
    if similar_snapshots:
        best_snapshot = max(similar_snapshots, key=lambda x: x["similarity"])
        print(f"\n\033[1;35mSNAPSHOT RECALL:\033[0m Found similar problem structure (similarity: {best_snapshot['similarity']:.2f})")

    groupchat = GroupChat(
        agents=agents,
        messages=[],
        max_round=12,
        speaker_selection_method=custom_speaker_selection,
        allow_repeat_speaker=True
    )

    manager = GroupChatManager(
        groupchat=groupchat,
        llm_config={
            "config_list": [{
                "model": AGENT_MODELS["metacognitive"],
                "api_key": OPENROUTER_API_KEY,
                "base_url": OPENROUTER_API_BASE
            }]
        },
    )

    user_proxy = UserProxyAgent(
        name="initiator",
        human_input_mode="NEVER",
        code_execution_config=False,
        is_termination_msg=lambda x: "TERMINATE" in x.get("content", "").upper(),
    )

    user_proxy.initiate_chat(manager, message=task, silent=True)

    # Extract final response from summarizer
    final_response = ""
    for msg in reversed(groupchat.messages):
        if msg["name"] == "summarizer_agent":
            final_response = msg["content"]
            break
    if not final_response:
        final_response = groupchat.messages[-1]["content"]

    agent_sequence = []
    for msg in groupchat.messages:
        if msg["name"] != "initiator" and msg["name"] not in agent_sequence:
            agent_sequence.append(msg["name"])

    perform_reflection(task, final_response, agent_sequence)

    verification = verify_solution(task, final_response)
    print(f"\n\033[1;35mRIGOROUS VERIFICATION REPORT:\033[0m\n{verification}")

    success = "correct" in verification.lower() or "valid" in verification.lower()
    motivation_system.update_from_outcome(task, success)

    return final_response

# ======================
# Answer Extraction
# ======================
def extract_final_answer(response: str) -> str:
    """
    Extracts the final answer from the response using pattern matching.

    Supports various formats like #### number, boxed, etc.

    Args:
        response (str): The response string.

    Returns:
        str: Extracted answer or empty string if not found.
    """
    # Try multiple patterns in order of specificity
    patterns = [
        # Pattern 1: Standard #### followed by number
        r'####\s*([-+]?[0-9,]*\.?[0-9]+)',
        # Pattern 2: Boxed answer format
        r'\\boxed\{([-+]?[0-9,]*\.?[0-9]+)\}',
        # Pattern 3: Any #### followed by anything and then a number
        r'####[^\d]*([-+]?[0-9,]*\.?[0-9]+)',
        # Pattern 4: Look for the final numerical answer anywhere in the last few lines
        r'([-+]?[0-9,]*\.?[0-9]+)(?=[^\d]*$)'
    ]

    for pattern in patterns:
        matches = re.findall(pattern, response)
        if matches:
            # Return last match and remove any commas
            result = matches[-1].replace(',', '').strip()
            # If it's a decimal number ending with .00, convert to integer
            if re.match(r'^\d+\.00$', result):
                result = result.split('.')[0]
            return result

    return ""

# ======================
# GSM8K Evaluation
# ======================
def load_gsm8k_dataset(start: int = 0, end: int = 15) -> List[Dict]:
    """
    Loads a subset of the GSM8K dataset from Hugging Face.

    Args:
        start (int, optional): Start index. Defaults to 0.
        end (int, optional): End index. Defaults to 15.

    Returns:
        List[Dict]: List of problem dictionaries with question and answer.
    """
    try:
        dataset = load_dataset("gsm8k", "main", split="test")
        dataset = dataset.select(range(start, end))
        return [{"question": item["question"], "answer": item["answer"]} for item in dataset]
    except Exception as e:
        print(f"Error loading GSM8K dataset: {e}")
        return []

def evaluate_on_gsm8k(dataset: List[Dict]):
    """
    Evaluates the framework on GSM8K problems.

    Processes each problem, extracts answers, checks correctness, and collects results.

    Args:
        dataset (List[Dict]): List of GSM8K problems.

    Returns:
        List[Dict]: List of evaluation results.
    """
    results = []

    for i, problem in enumerate(dataset, 1):
        question = problem["question"]
        # Extract ground truth answer from GSM8K format
        ground_truth = problem["answer"].split("####")[-1].strip()

        print(f"\n\033[1;34mPROBLEM {i}: {question}\033[0m")
        print("\033[90m" + "─" * 80 + "\033[0m")

        try:
            start_time = time.time()
            response = process_task(question)
            elapsed = time.time() - start_time

            predicted_answer = extract_final_answer(response)
            correct = predicted_answer == ground_truth

            results.append({
                "problem": question,
                "response": response,
                "predicted": predicted_answer,
                "ground_truth": ground_truth,
                "correct": correct,
                "time": elapsed
            })

            print(f"\n\033[1;32mPREDICTED ANSWER: {predicted_answer}\033[0m")
            print(f"\033[1;33mGROUND TRUTH: {ground_truth}\033[0m")
            print(f"\033[1;36mCORRECT: {'✓' if correct else '✗'}\033[0m")
            print(f"Time: {elapsed:.2f} seconds")
            print("\033[90m" + "─" * 80 + "\033[0m")

        except Exception as e:
            print(f"\n\033[1;31mERROR PROCESSING PROBLEM {i}: {e}\033[0m")
            results.append({
                "problem": question,
                "error": str(e),
                "time": -1
            })

    return results

# ======================
# Performance Reporting
# ======================
def generate_performance_report(results: List[Dict]):
    """
    Generates a detailed performance report from evaluation results.

    Includes accuracy, average time, and per-problem details.

    Args:
        results (List[Dict]): List of evaluation results.

    Returns:
        str: Formatted report string.
    """
    correct_count = sum(1 for r in results if r.get("correct", False))
    total = len(results)
    accuracy = correct_count / total if total > 0 else 0

    valid_times = [r["time"] for r in results if r.get("time", -1) > 0]
    avg_time = sum(valid_times) / len(valid_times) if valid_times else 0

    report = "\n" + "="*80 + "\n"
    report += " GSM8K EVALUATION REPORT ".center(80) + "\n"
    report += "="*80 + "\n"

    report += f"\n\033[1;36mACCURACY: {accuracy*100:.2f}% ({correct_count}/{total})\033[0m\n"
    report += f"\033[1;36mAVERAGE INFERENCE TIME: {avg_time:.2f} seconds\033[0m\n"

    report += "\n\033[1;35mDETAILED RESULTS:\033[0m\n"
    for i, result in enumerate(results, 1):
        report += f"\nPROBLEM {i}:\n"
        report += f"  Question: {result['problem']}\n"

        if "error" in result:
            report += f"  \033[1;31mERROR: {result['error']}\033[0m\n"
        else:
            report += f"  Response: {result['response'][:200]}...\n"
            report += f"  Predicted: {result['predicted']}\n"
            report += f"  Ground Truth: {result['ground_truth']}\n"
            status = "\033[1;32mCORRECT\033[0m" if result['correct'] else "\033[1;31mINCORRECT\033[0m"
            report += f"  Status: {status}\n"
            report += f"  Time: {result['time']:.2f} seconds\n"

    report += "\n" + "="*80 + "\n"
    return report

# ======================
# Main Execution
# ======================
def main():
    """
    Main execution function.

    Loads dataset, pre-stores template, evaluates on GSM8K, generates reports, and cleans up.
    """
    try:
        start_time = datetime.datetime.now()

        # Load GSM8K dataset from Hugging Face
        start = 0
        end = 50
        gsm8k_data = load_gsm8k_dataset(start=start, end=end)

        # Pre-store some structural templates for math problems
        math_template = snapshot_memory.create_snapshot(
            "If Alice has 5 apples and Bob gives her 3 more, how many apples does Alice have? #### 8",
            "math_word_problem"
        )
        snapshot_memory.create_snapshot(
            "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May? #### 72",
            "math_word_problem"
        )
        snapshot_memory.create_snapshot(
            "Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn? #### 10",
            "math_word_problem"
        )
        snapshot_memory.create_snapshot(
            "Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet? #### 5",
            "math_word_problem"
        )
        snapshot_memory.create_snapshot(
            "Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read? #### 42",
            "math_word_problem"
        )
        snapshot_memory.create_snapshot(
            "James writes a 3-page letter to 2 different friends twice a week. How many pages does he write a year? #### 624",
            "math_word_problem"
        )

        # Evaluate on GSM8K problems
        results = evaluate_on_gsm8k(gsm8k_data)

        # Generate performance report
        print(generate_performance_report(results))

        # Generate snapshot memory report
        print(f"\n\033[1;35mSNAPSHOT MEMORY USAGE:\033[0m")
        print(f"Stored snapshots: {len(snapshot_memory.snapshots)}")
        if snapshot_memory.snapshots:
            print("Recent snapshots:")
            for i, (key, snapshot) in enumerate(list(snapshot_memory.snapshots.items())[-3:]):
                print(f"  {i+1}. Type: {snapshot['structure_type']}")
                print(f"     Features: {snapshot['structural_features']}")

        # Calculate total execution time
        total_time = (datetime.datetime.now() - start_time).total_seconds()
        print(f"Total execution time: {total_time:.2f} seconds")

        return results
    finally:
        # Clean up resources
        spontaneous_agent.stop()

if __name__ == "__main__":
    main()

Snapshot Memory System initialized
Spontaneous Thought Agent started

PROBLEM 1: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
────────────────────────────────────────────────────────────────────────────────

Next speaker: router_agent


>>>>>>>> USING AUTO REPLY...
[autogen.oai.client: 09-19 18:38:09] {695} WARNING - Model meta-llama/llama-3.3-70b-instruct is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


router_agent (to chat_manager):

language,logic,world

--------------------------------------------------------------------------------

Next speaker: language_agent


>>>>>>>> USING AUTO REPLY...
[autogen.oai.client: 09-19 18:38:19] {695} WARNING - Model qwen/qwen-2.5-72b-instruct is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


language_agent (to chat_manager):

### Summary:
Verdict: Janet makes $18 every day at the farmers' market. Method Sketch: Calculate the total eggs laid, subtract eggs used, and multiply the remaining eggs by the selling price.

### Detailed Analysis:
1. **Total Eggs Laid**: Janet’s ducks lay 16 eggs per day.
   - Justification: Given in the problem statement.

2. **Eggs Used for Breakfast**: Janet eats 3 eggs for breakfast every morning.
   - Justification: Given in the problem statement.

3. **Eggs Used for Muffins**: Janet bakes muffins for her friends every day using 4 eggs.
   - Justification: Given in the problem statement.

4. **Total Eggs Used**: Calculate the total number of eggs used for breakfast and muffins.
   - Calculation: 3 (breakfast) + 4 (muffins) = 7 eggs.
   - Justification: Sum of eggs used for both activities.

5. **Remaining Eggs**: Subtract the total eggs used from the total eggs laid.
   - Calculation: 16 (total eggs) - 7 (used eggs) = 9 eggs.
   - Justification

logic_agent (to chat_manager):

### Summary:
Verdict: Janet makes $18 every day at the farmers' market. Method Sketch: Calculate the total eggs laid, subtract eggs used, and multiply the remaining eggs by the selling price.

### Detailed Analysis:
1. **Total Eggs Laid**: Janet’s ducks lay 16 eggs per day.
   - Justification: Given in the problem statement.

2. **Eggs Used for Breakfast**: Janet eats 3 eggs for breakfast every morning.
   - Justification: Given in the problem statement.

3. **Eggs Used for Muffins**: Janet bakes muffins for her friends every day using 4 eggs.
   - Justification: Given in the problem statement.

4. **Total Eggs Used**: Calculate the total number of eggs used for breakfast and muffins.
   - Calculation: 3 (breakfast) + 4 (muffins) = 7 eggs.
   - Justification: Sum of eggs used for both activities.

5. **Remaining Eggs**: Subtract the total eggs used from the total eggs laid.
   - Calculation: 16 (total eggs) - 7 (used eggs) = 9 eggs.
   - Justification: R

world_agent (to chat_manager):

### Summary:
Verdict: Janet makes $18 every day at the farmers' market. Method Sketch: Calculate the total eggs laid, subtract eggs used, and multiply the remaining eggs by the selling price.

### Detailed Analysis:
1. **Total Eggs Laid**: Janet’s ducks lay 16 eggs per day.
   - Justification: Given in the problem statement.

2. **Eggs Used for Breakfast**: Janet eats 3 eggs for breakfast every morning.
   - Justification: Given in the problem statement.

3. **Eggs Used for Muffins**: Janet bakes muffins for her friends every day using 4 eggs.
   - Justification: Given in the problem statement.

4. **Total Eggs Used**: Calculate the total number of eggs used for breakfast and muffins.
   - Calculation: 3 (breakfast) + 4 (muffins) = 7 eggs.
   - Justification: Sum of eggs used for both activities.

5. **Remaining Eggs**: Subtract the total eggs used from the total eggs laid.
   - Calculation: 16 (total eggs) - 7 (used eggs) = 9 eggs.
   - Justification: R

...

router_agent (to chat_manager):

language,logic

--------------------------------------------------------------------------------

Next speaker: language_agent


>>>>>>>> USING AUTO REPLY...
[autogen.oai.client: 09-19 19:14:20] {695} WARNING - Model qwen/qwen-2.5-72b-instruct is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


language_agent (to chat_manager):

### Summary
Verdict: The total number of unoccupied units is 30. Method Sketch: Calculate the total number of units, determine the number of occupied units, and subtract to find the unoccupied units.

### Detailed Analysis

1. **Calculate Total Number of Units:**
   - The building has 15 floors.
   - Each floor contains 8 units.
   - Total units = 15 floors × 8 units/floor = 120 units.
   - Justification: Multiplication to find the total number of units in the building.

2. **Determine Number of Occupied Units:**
   - 3/4 of the building is occupied.
   - Number of occupied units = (3/4) × 120 units = 90 units.
   - Justification: Fraction multiplication to find the number of occupied units.

3. **Calculate Number of Unoccupied Units:**
   - Total units = 120.
   - Occupied units = 90.
   - Unoccupied units = Total units - Occupied units = 120 - 90 = 30 units.
   - Justification: Subtraction to find the number of unoccupied units.

### Final Answer
##

logic_agent (to chat_manager):

#### 30

### Summary
Verdict: The total number of unoccupied units is 30. Method Sketch: Calculate the total number of units, determine the number of occupied units, and subtract to find the unoccupied units.

### Detailed Analysis

1. **Calculate Total Number of Units:**
   - The building has 15 floors.
   - Each floor contains 8 units.
   - Total units = 15 floors × 8 units/floor = 120 units.
   - Justification: Multiplication to find the total number of units in the building.

2. **Determine Number of Occupied Units:**
   - 3/4 of the building is occupied.
   - Number of occupied units = (3/4) × 120 units = 90 units.
   - Justification: Fraction multiplication to find the number of occupied units.

3. **Calculate Number of Unoccupied Units:**
   - Total units = 120.
   - Occupied units = 90.
   - Unoccupied units = Total units - Occupied units = 120 - 90 = 30 units.
   - Justification: Subtraction to find the number of unoccupied units.

### Final Ans

summarizer_agent (to chat_manager):

The calculation of unoccupied units in the building is based on a straightforward, step-by-step analysis combining logical mathematical operations with a clear understanding of the problem's parameters.

Firstly, the **total number of units** in the building is found by multiplying the number of floors by the number of units per floor, which is 15 * 8 = 120 units. This step is logically sound and mathematically accurate, providing a solid foundation for further calculations.

Secondly, determining the **number of occupied units** involves multiplying the total number of units by the fraction of the building that is occupied, which is 3/4 * 120 = 90 units. This calculation correctly applies the concept of fractions to represent the portion of the building that is occupied.

Lastly, finding the **number of unoccupied units** is a simple matter of subtracting the number of occupied units from the total number of units, yielding 120 - 90 = 30 units. Thi

metacognitive_agent (to chat_manager):

### Verification Report
- Critical Errors: None.
- Justification Gaps: The detailed step-by-step explanation provided covers all necessary mathematical steps and logical reasoning, leaving no significant gaps in justification.
- Overall Assessment: The analysis is thorough, and the conclusion is correctly reached through a series of logical and mathematically sound steps.
- Format: The final answer is correctly formatted as requested.

Given the comprehensive nature of the analysis and the absence of any critical errors or significant justification gaps, the conclusion that there are 30 unoccupied units in the building is well-supported and accurately presented.

Output: TERMINATE

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (8f053aa8-d4df-46b7-b2c5-3495ca65193a): No next speaker selected

RIGOROUS VERIFICATION REPORT:
### Verification Report ###

- **Critical Errors:** None. The solu